In [293]:
from ryPat import *
from ryPat_Optimization_01 import *
%matplotlib qt

f([x,y])

# 這是 objective function
# 目前本程式處理 2-d 函數 f(s)
# 其中
# s= [x,y]= sm.symbols('x,y')
#
# 在本章中 (x,y) 濃縮在 s 裡面，
# 這是 為了 scipy.optimize 的要求

def f(s):
    x, y= s
    
    z=  (x-5)**2 + (y-5)**2
    #z= -2**(-z/25)
    
    return z

# constraint函數, 必須同時有 x,y 項，
# 允許用 1e-10 當作 近似 0 的係數，
# 但絕大多數時候不需要，因為可以用 xyBounds 達成相同的目的
def c1(s):
    x,y= s
    #z= x + y*1e-3 +10  # 模擬 x + 10 > =0
    z= x + y - 1        # x + y - 1 >= 0
    return z

def c2(s):
    x,y= s
    #z= x*1e-3 +y +10   # 模擬 y + 10 > =0
    z= -x + 2*y -2      # -x + 2*y - 2 >= 0
    return z

def c3(s):
    x,y= s
    #z= -(x + 3*y -9)
    #z= x + y + 20
    #z= -x - 3*y +30
    ## 試試看2次函數當作constraints
    z= -x**2 - y +10  # -x**2 - y +10 >=0
    return z

Constraints= [
    {'fun':c1, 'type':'ineq'},
    {'fun':c2, 'type':'ineq'},
    {'fun':c3, 'type':'ineq'},
]

xyBounds= [(-5, 5),  # xmin, xmax
           (-5, 5)   # ymin, ymax
           ]

#ryOptimization(f,c1,c2,c3, xyBounds)

'''
def ryOptimization(
        f= f, 
        c1= c1, 
        c2= c2, 
        c3= c3, 
        xyBounds= xyBounds):
'''
# 接下來我們回到 平面上來細部拆分

#
# Objective function s.t. Constraints
#
x,y= sm.symbols('x,y')

# sympy function --> numpy function
z= sm.lambdify([x,y],f([x,y])) 

xx= np.linspace(-10,10,1001)
yy= np.linspace(-10,10,1001)
zz= z(xx,yy)

xm, ym= np.meshgrid(xx,yy)
zm= z(xm,ym)
zz.shape, zm.shape



#
# plot constraints 
#

#  Constraints 
# 如何自動從 c1, c2, c3 撈出來？？
# 需要 sympy.solve()
#
y1= sm.solve(
        c1([x,y]),
        y)[0]
y2= sm.solve(
        c2([x,y]),
        y)[0]
y3= sm.solve(
        c3([x,y]),
        y)[0]

# sympy 解方程式之後，仍然需要numpy

y1= sm.lambdify(x, y1)
y2= sm.lambdify(x, y2)
y3= sm.lambdify(x, y3)

yy1= y1(xx)
yy2= y2(xx)
yy3= y3(xx)
#-----------------------

#
#  平行x軸,y軸 界線 (xyBounds) 可以如下製造出來 
#
[(xmin,xmax),(ymin,ymax)]= xyBounds
           
y_ymin= ymin *np.ones_like(xx)
y_ymax= ymax *np.ones_like(xx)
x_xmin= xmin *np.ones_like(yy)
x_xmax= xmax *np.ones_like(yy)

#
# 坐標軸 (x軸,y軸) 可以如下製造出來 
#
y_0= np.zeros_like(xx)
x_0= np.zeros_like(yy)

ax= pl.axes(xlim=(-10,10),ylim=(-10,10))#, projection='3d')

ax.contour(xm,ym,zm, 100, cmap='rainbow')

ax.plot(xx,yy1,'r',
        xx,yy2,'g',
        xx,yy3,'b',
        linestyle='--'
       )    





#
# 畫坐標軸 (x軸,y軸) 
#
ax.plot(xx, y_0,'w-',
        x_0,yy, 'w-',
        linewidth= 5,
        alpha= 0.5 
       )

#
#  畫 xyBounds (平行x軸,y軸) 
#
ax.plot(xx,     y_ymin,
        xx,     y_ymax,
        x_xmin, yy, 
        x_xmax, yy, 
        linewidth= 1,
        linestyle= '--',
        color= 'gray'
       )

# 把 符合限制條件的區域標示出來 (塗滿灰色)

ax.fill_between(xx, yy3, yy1,                
                where= (yy3 >= yy1)&(yy1>=yy2),
                alpha=0.50, 
                color='gray', 
                interpolate=True
               )
ax.fill_between(xx, yy3, yy2,                
                where= (yy3 >= yy2)&(yy2>=yy1),
                alpha=0.50, 
                color='gray', 
                interpolate=True
               )
ax.fill_between(xx, yy3, y_ymax,                
                where= (yy3 >= y_ymax),
                alpha=0.50, 
                color='gray', 
                interpolate=True
               )

# 在這個區域內找最小 f([x,y])
import scipy.optimize as sopt

x0= [0, 0]

opt= sopt.minimize(
    f,
    x0,
    #method= 'SLSQP',
    bounds= xyBounds,
    constraints=     Constraints
)

opt

#
# 畫 最佳點 opt.x 
#

xopt= opt.x[0]
yopt= opt.x[1]
fopt= opt.fun

ax.scatter(
    x= xopt,
    y= yopt, 
    color= 'magenta',
    marker= 's'    
)

ax.text(
    x= xopt, 
    y= yopt,
    s= f'{xopt:.3f},{yopt:.3f}'
)


fLtx=  sm.latex(f([x,y]))
c1Ltx= sm.latex(c1([x,y]))
c2Ltx= sm.latex(c2([x,y]))
c3Ltx= sm.latex(c3([x,y]))

titleStr= f'''
Optimize:      f(x,y)= ${fLtx}$
Subject to:    Constraints and Bounds
opt= [x= {xopt:.3f}, y= {yopt:.3f}; f= {fopt:.3f}]
'''

ax.set_title(titleStr)
ax.set_xlabel('x')
ax.set_ylabel('y')

infoStr= f'''
$Objective: f(x,y)= {fLtx}$
$c_1(x,y)= {c1Ltx} ≥ 0$ 
$c_2(x,y)= {c2Ltx} ≥ 0$ 
$c_3(x,y)= {c3Ltx} ≥ 0$
$x \in [{xmin},{xmax}]$ 
$y \in [{ymin},{ymax}]$
opt= [x= {xopt:.3f}, y= {yopt:.3f}; f= {fopt:.3f}]
'''
ax.text(
    x= -10, 
    y= -10,
    s= infoStr
)

Text(-10, -10, '\n$Objective: f(x,y)= \\left(x - 5\\right)^{2} + \\left(y - 5\\right)^{2}$\n$c_1(x,y)= x + y - 1 ≥ 0$ \n$c_2(x,y)= - x + 2 y - 2 ≥ 0$ \n$c_3(x,y)= - x^{2} - y + 10 ≥ 0$\n$x \\in [-5,5]$ \n$y \\in [-5,5]$\nopt= [x= 2.358, y= 4.440; f= 7.294]\n')

$ x \ge y$

In [219]:
fLtx=  sm.latex(f([x,y]))
c1Ltx= sm.latex(c1([x,y]))
c2Ltx= sm.latex(c2([x,y]))
c3Ltx= sm.latex(c3([x,y]))


Traceback (most recent call last):
  File "M:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mathtext.py", line 2609, in parse
    result = self._expression.parseString(s)
  File "M:\ProgramData\Anaconda3\lib\site-packages\pyparsing.py", line 1955, in parseString
    raise exc
  File "M:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mathtext.py", line 2767, in unknown_symbol
    raise ParseFatalException(s, loc, "Unknown symbol: %s" % c)
pyparsing.ParseFatalException: Unknown symbol: \ge, found '\'  (at char 10), (line:1, col:11)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "M:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_qt5.py", line 480, in _draw_idle
    self.draw()
  File "M:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 407, in draw
    self.figure.draw(self.renderer)
  File "M:\ProgramData\Anaconda3\lib\site-packages\matplotlib\artist.py", line 

In [30]:

#
# 設定 2軸 顯示範圍
#
ax.set_xlim([-10,10])
ax.set_ylim([-10,10])


#
# 再加碼 ，把 constraints 升至 立體的曲面 
#

xx= np.linspace(-10,10,1001)

#yy1= 3*xx+6
#yy2= (-xx+4)/2
#yy3= xx*0 -3

zz1= f([xx,yy1])
zz2= f([xx,yy2])
zz3= f([xx,yy3])

ax.plot3D(xx,yy1,zz1, color='r', linestyle='-')
ax.plot3D(xx,yy2,zz2, color='g', linestyle='-')
ax.plot3D(xx,yy3,zz3, color='b', linestyle='-')

pl.show()

Traceback (most recent call last):
  File "M:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "M:\ProgramData\Anaconda3\lib\site-packages\mpl_toolkits\mplot3d\axes3d.py", line 1137, in _button_release
    toolbar = getattr(self.figure.canvas, "toolbar")
AttributeError: 'NoneType' object has no attribute 'canvas'
Traceback (most recent call last):
  File "M:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "M:\ProgramData\Anaconda3\lib\site-packages\mpl_toolkits\mplot3d\axes3d.py", line 1137, in _button_release
    toolbar = getattr(self.figure.canvas, "toolbar")
AttributeError: 'NoneType' object has no attribute 'canvas'
Traceback (most recent call last):
  File "M:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "M:\ProgramData\Anaconda3\lib\site-packages\mpl

In [25]:
ryOptimization()

TypeError: cannot unpack non-iterable Axes3DSubplot object

In [24]:
'''
plt.fill_between(overs, run_rate, required_run_rate,
                 where=(z1 < z2),
                 alpha=0.30, color='red', interpolate=True)
plt.fill_between(overs, run_rate, 8.9,
                 where=(z1 >= z2),
                 alpha=0.30, color='green', interpolate=True)
'''

TypeError: cannot unpack non-iterable Axes3DSubplot object